In [ ]:
import pathlib
import json
import helpers
from helpers import ai, apple
from bs4 import BeautifulSoup

from openai import OpenAI

In [ ]:
NBS_DIR = pathlib.Path().resolve().parent
BASE_DIR = NBS_DIR
DATASET_DIR = BASE_DIR / "dataset"

In [ ]:
for path in list(DATASET_DIR.glob("**/**/pred.json"))[:10]:
    post_dir = path.parent
    print(post_dir)
    post_id = post_dir.name
    podcast_dir = post_dir / "podcasts"
    podcast_dir.mkdir(parents=True, exist_ok=True)
    pred_data = json.loads(path.read_text())
    mp3s = []
    if isinstance(pred_data, dict):
        try:
            keywords = pred_data['keywords']
        except:
            keywords = []
        lookup_list = []
        for keyword in keywords:
            val = keyword.get('value')
            rank = int(keyword.get('rank'))
            if rank == 1: # top ranked keyword
                lookup_list.append(val)
        for keyword in lookup_list:
            if not keyword:
                continue
            print('Searching for podcasts', keyword)
            podcast_results = apple.perform_search(search_term=keyword)
            for podcast in podcast_results:
                print("Saving details for", podcast.get('trackName'))
                track_id = podcast.get('trackId')
                track_output_dir = podcast_dir / f'{track_id}'
                track_output_dir.mkdir(parents=True, exist_ok=True)
                track_output_data_path = track_output_dir / "podcast.json"
                track_output_data_path.write_text(json.dumps(podcast, indent=4))